# Reconciliation of existing CHO metabolic reconstructions
This notebook comprises all the steps to reconcile and merge the existing metabolic reconstruction in CHO cells

### Datasets generation and merge

In [1]:
# Libraries import
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from requests_html import HTMLSession

from tqdm.notebook import tqdm

In [ ]:
#Read excel files and create the dfs

#Camels CHO_DG44 metabolic reconstruction
camel_df = pd.read_excel('../Data/1-s2.0-S1096717618302258-mmc1.xlsx', header = 1)

#Hefzi's iCHO1766 metabolic reconstruction
hefzi_df = pd.read_excel('../Data/hefzi_final.xlsx')

#Foudaliha's iCHO2101 metabolic reconstruction
fouladiha_df = pd.read_excel('../Data/10529_2020_3021_MOESM1_ESM.xlsx', 'Supplementary Table 10', header = 1)

#Yeo's iCHO2291 metabolic reconstruction
iCHO2291 = pd.read_excel('../Data/iCHO2291_final.xlsx')

In [ ]:
#Standarization of the columns names
camel_df.rename(columns = {'Reaction ID':'Reaction', 'Initial reaction in model':'Reaction Formula', 'Reaction name':'Reaction Name', 'Justification':'Curation Notes'}, inplace = True)
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula'}, inplace = True)

# Addition of tag columns for organizational purposes
camel_df.insert(loc=0, column='cam', value='X')
camel_df.insert(loc=1, column='hef', value=np.nan)
camel_df.insert(loc=2, column='fou', value=np.nan)
camel_df.insert(loc=3, column='yeo', value=np.nan)

hefzi_df.insert(loc=0, column='cam', value=np.nan)
hefzi_df.insert(loc=1, column='hef', value='X')
hefzi_df.insert(loc=2, column='fou', value=np.nan)
hefzi_df.insert(loc=3, column='yeo', value=np.nan)

fouladiha_df.insert(loc=0, column='cam', value=np.nan)
fouladiha_df.insert(loc=1, column='hef', value=np.nan)
fouladiha_df.insert(loc=2, column='fou', value='X')
fouladiha_df.insert(loc=3, column='yeo', value=np.nan)

iCHO2291.insert(loc=0, column='cam', value=np.nan)
iCHO2291.insert(loc=1, column='hef', value=np.nan)
iCHO2291.insert(loc=2, column='fou', value=np.nan)
iCHO2291.insert(loc=3, column='yeo', value='X')


fouladiha_df

In [ ]:
# Generate cols list with the column names from all datasets
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+iCHO2291.columns.to_list()+camel_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

cols

In [ ]:
def add_col(df):
    '''
    This function adds the columns from the cols list 
    to an existing df doesnt contain those columns
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [ ]:
# Unify columns for all datasets
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
iCHO2291 = add_col(iCHO2291)
camel_df = add_col(camel_df)

iCHO2291

In [ ]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
iCHO2291 = iCHO2291[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
hefzi_df = hefzi_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
camel_df = camel_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]

camel_df['Reaction'] = camel_df['Reaction'].str.strip()

In [ ]:
# Merge all the dfs into a unified df
all_dfs = pd.concat([camel_df, hefzi_df, fouladiha_df, iCHO2291])
all_dfs = all_dfs.reset_index(drop = True)
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace('_cho', '')

all_dfs #20940 rows/reactions (many of them repeated)

### Different ways of grouping the data

In [ ]:
all_dfs1 = all_dfs.groupby(['Reaction'], group_keys=True).apply(lambda x: x.fillna(x.ffill()))
#all_dfs1 = all_dfs.groupby(['Reaction','GPR'], group_keys=True).first()

all_dfs1.to_excel('../Data/all_dfs1.xlsx')

In [ ]:
# Group the entire dataset by reaction name 
all_dfs1 = all_dfs.groupby(['Reaction'], group_keys=True).apply(lambda x: x)

# Save merged datasets as Excel file
all_dfs1.to_excel('../Data/all_dfs1.xlsx')

all_dfs1

In [ ]:
all_dfs2 = all_dfs.groupby('Reaction').first()
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace('[','_')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(']','')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' => ',' --> ')
all_dfs2.to_excel('../Data/all_dfs2.xlsx')
all_dfs2

In [ ]:
iCHOxxxx = pd.read_excel('../Data/all_dfs2_mod.xlsx')
iCHOxxxx

## Compile dataframe into a model using the cobrapy package

In [2]:
import cobra

iCHOxxxx = pd.read_excel('../Data/all_dfs2_mod.xlsx')

model = cobra.Model("iCHOxxxx")
lr = []

for _, row in tqdm(iCHOxxxx.reset_index().iterrows()):
    r = cobra.Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

0it [00:00, ?it/s]

Name,iCHOxxxx
Memory address,1204762c0
Number of metabolites,0
Number of reactions,11283
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [3]:
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(iCHOxxxx['Reaction Formula'][i])
    print(r)


  0%|          | 0/11283 [00:00<?, ?it/s]

unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
10FTHF5GLUtl: 10fthf5glu_c --> 10fthf5glu_l
unknown metabolite '10fthf5glu_m' created
10FTHF5GLUtm: 10fthf5glu_m --> 10fthf5glu_c
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
10FTHF6GLUtl: 10fthf6glu_c --> 10fthf6glu_l
unknown metabolite '10fthf6glu_m' created
10FTHF6GLUtm: 10fthf6glu_m --> 10fthf6glu_c
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
10FTHF7GLUtl: 10fthf7glu_c --> 10fthf7glu_l
unknown metabolite '10fthf7glu_m' created
10FTHF7GLUtm: 10fthf7glu_m --> 10fthf7glu_c
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
10FTHFtl: 10fthf_c <=> 10fthf_l
unknown metabolite '10fthf_m' created
10FTHFtm: 10fthf_c <=> 10fthf_m
unknown metabolite '11docrtsl_c' created
unknown metabolite '11docrtsl_m' created
11DOCRTSLtm: 11docrtsl_c <=> 11docrtsl_m
unknown metabolite '11docrtsl_r' created
11DOCRTSLtr: 11d

4HGLSDm: 4hglusa_m + h2o_m + nad_m <=> e4hglu_m + 2.0 h_m + nadh_m
unknown metabolite '4hoxpacd_c' created
unknown metabolite '4hphac_c' created
4HOXPACDOX: 4hoxpacd_c + h2o_c + nad_c --> 4hphac_c + 2.0 h_c + nadh_c
4HOXPACDOX_NADP_: 4hoxpacd_c + h2o_c + nadp_c <=> 4hphac_c + 2.0 h_c + nadph_c
4HOXPACDOXy: 4hoxpacd_c + h2o_c + nadp_c --> 4hphac_c + 2.0 h_c + nadph_c
unknown metabolite '4hpro_LT_c' created
unknown metabolite '1p3h5c_c' created
4HPRO5OARGm_c: 4hpro_LT_c + nadp_c <=> 1p3h5c_c + 2.0 h_c + nadph_c
unknown metabolite 'ala_L_e' created
unknown metabolite '4hpro_LT_e' created
unknown metabolite 'ala_L_c' created
4HPROLTASCT1: 4hpro_LT_c + ala_L_e + na1_e --> 4hpro_LT_e + ala_L_c + na1_c
unknown metabolite '4hpro_LT_m' created
4HPRO_LTte: 4hpro_LT_e <=> 4hpro_LT_m
unknown metabolite 'pydam_c' created
unknown metabolite 'pydx_c' created
4M2OPLOXRD: h2o_c + o2_c + pydam_c --> h2o2_c + nh4_c + pydx_c
unknown metabolite '4mop_m' created
4MOPLPAM3MBD_m: 4mop_m + h_m + lpam_m --> HC0

unknown metabolite 'lnlncg_c' created
unknown metabolite 'lnlncg_e' created
ABC_TCDBa9: atp_c + h2o_c + lnlncg_c --> adp_c + h_c + lnlncg_e + pi_c
unknown metabolite 'h2o_r' created
unknown metabolite 'atp_r' created
unknown metabolite 'bildglcur_r' created
unknown metabolite 'bildglcur_e' created
unknown metabolite 'adp_r' created
unknown metabolite 'pi_r' created
ABC_TCDBb: atp_r + bildglcur_r + h2o_r --> adp_r + bildglcur_e + h_r + pi_r
unknown metabolite 'chsterol_r' created
unknown metabolite 'chsterol_e' created
ABC_TCDBc: atp_r + chsterol_r + h2o_r --> adp_r + chsterol_e + h_r + pi_r
unknown metabolite 'chedxch_c' created
unknown metabolite 'chedxch_e' created
ABC_TCDBd: atp_c + chedxch_c + h2o_c --> adp_c + chedxch_e + h_c + pi_c
unknown metabolite 'dgchol_c' created
unknown metabolite 'dgchol_e' created
ABC_TCDBe1: atp_c + dgchol_c + h2o_c --> adp_c + dgchol_e + h_c + pi_c
unknown metabolite 'tdchola_c' created
unknown metabolite 'tdchola_e' created
ABC_TCDBe2: atp_c + h2o_c +

ACCOACm: accoa_m + atp_m + hco3_m --> adp_m + h_m + malcoa_m + pi_m
ACCOAL: atp_c + coa_c + ppa_c --> adp_c + pi_c + ppcoa_c
unknown metabolite '5apade_c' created
ACCOAL2: ac_c + atp_c + h_c --> 5apade_c + ppi_c
unknown metabolite 'ppa_m' created
ACCOALm: atp_m + coa_m + ppa_m --> amp_m + ppcoa_m + ppi_m
unknown metabolite 'accoa_g' created
ACCOAgt: accoa_c <=> accoa_g
unknown metabolite 'accoa_n' created
ACCOAtn: accoa_c <=> accoa_n
unknown metabolite 'accoa_r' created
ACCOAtr: accoa_c <=> accoa_r
unknown metabolite 'acetone_e' created
unknown metabolite 'acetone_c' created
ACETONEt2: acetone_e + h_e <=> acetone_c + h_c
unknown metabolite 'acetone_m' created
ACETONEt2m: acetone_c + h_c <=> acetone_m + h_m
ACFORte: ac_e + for_c <=> ac_c + for_e
unknown metabolite 'acgagbside_cho_c' created
unknown metabolite 'acgagbside_cho_g' created
ACGAGBSIDEtg: acgagbside_cho_c <=> acgagbside_cho_g
unknown metabolite 'acgagbside_cho_l' created
ACGAGBSIDEtl: acgagbside_cho_c <=> acgagbside_cho_l
unk

...

r1721: na1_e + ser_L_e + trp_L_c --> na1_c + ser_L_c + trp_L_e
r1722: na1_e + phe_L_c + ser_L_e --> na1_c + phe_L_e + ser_L_c
r1723: na1_e + ser_L_e + tyr_L_c --> na1_c + ser_L_c + tyr_L_e
r1725: leu_L_c + na1_e + ser_L_e --> leu_L_e + na1_c + ser_L_c
r1726: na1_e + pro_L_c + ser_L_e --> na1_c + pro_L_e + ser_L_c
r1728: na1_e + ser_L_e + val_L_c --> na1_c + ser_L_c + val_L_e
r1730: hom_L_c + na1_e + ser_L_e --> hom_L_e + na1_c + ser_L_c
r1731: ile_L_c + na1_e + ser_L_e --> ile_L_e + na1_c + ser_L_c
r1732: gly_c + met_L_e + na1_e --> gly_e + met_L_c + na1_c
r1733: ala_L_c + met_L_e + na1_e --> ala_L_e + met_L_c + na1_c
r1734: gln_L_c + met_L_e + na1_e --> gln_L_e + met_L_c + na1_c
r1735: met_L_e + na1_e + ser_L_c --> met_L_c + na1_c + ser_L_e
r1736: met_L_e + na1_e + trp_L_c --> met_L_c + na1_c + trp_L_e
r1737: met_L_e + na1_e + phe_L_c --> met_L_c + na1_c + phe_L_e
r1738: met_L_e + na1_e + tyr_L_c --> met_L_c + na1_c + tyr_L_e
r1739: cys_L_c + met_L_e + na1_e --> cys_L_e + met_L_c + na

r1865: hom_L_c + na1_e + val_L_e --> hom_L_e + na1_c + val_L_c
r1866: ile_L_c + na1_e + val_L_e --> ile_L_e + na1_c + val_L_c
r1867: gly_c + na1_e + thr_L_e --> gly_e + na1_c + thr_L_c
r1871: met_L_c + na1_e + thr_L_e --> met_L_e + na1_c + thr_L_c
r1872: na1_e + thr_L_e + trp_L_c --> na1_c + thr_L_c + trp_L_e
r1873: na1_e + phe_L_c + thr_L_e --> na1_c + phe_L_e + thr_L_c
r1874: na1_e + thr_L_e + tyr_L_c --> na1_c + thr_L_c + tyr_L_e
r1876: leu_L_c + na1_e + thr_L_e --> leu_L_e + na1_c + thr_L_c
r1877: na1_e + pro_L_c + thr_L_e --> na1_c + pro_L_e + thr_L_c
r1879: na1_e + thr_L_e + val_L_c --> na1_c + thr_L_c + val_L_e
r1880: hom_L_c + na1_e + thr_L_e --> hom_L_e + na1_c + thr_L_c
r1881: ile_L_c + na1_e + thr_L_e --> ile_L_e + na1_c + thr_L_c
r1882: gly_c + hom_L_e + na1_e --> gly_e + hom_L_c + na1_c
r1883: ala_L_c + hom_L_e + na1_e --> ala_L_e + hom_L_c + na1_c
r1884: gln_L_c + hom_L_e + na1_e --> gln_L_e + hom_L_c + na1_c
r1885: hom_L_e + na1_e + ser_L_c --> hom_L_c + na1_c + ser_L_e


r2012: arg_L_e + met_L_c --> arg_L_c + met_L_e
r2013: arg_L_e + trp_L_c --> arg_L_c + trp_L_e
r2014: arg_L_e + phe_L_c --> arg_L_c + phe_L_e
r2015: arg_L_e + tyr_L_c --> arg_L_c + tyr_L_e
r2016: arg_L_e + cys_L_c --> arg_L_c + cys_L_e
r2017: arg_L_e + leu_L_c --> arg_L_c + leu_L_e
r2018: arg_L_e + pro_L_c --> arg_L_c + pro_L_e
r2019: arg_L_e + asn_L_c --> arg_L_c + asn_L_e
r2020: arg_L_e + val_L_c --> arg_L_c + val_L_e
r2021: arg_L_e + thr_L_c --> arg_L_c + thr_L_e
r2022: arg_L_e + hom_L_c --> arg_L_c + hom_L_e
r2023: arg_L_e + ile_L_c --> arg_L_c + ile_L_e
r2024: gly_c + orn_e --> gly_e + orn_c
r2026: gln_L_c + orn_e --> gln_L_e + orn_c
r2027: orn_e + ser_L_c --> orn_c + ser_L_e
r2028: met_L_c + orn_e --> met_L_e + orn_c
r2029: orn_e + trp_L_c --> orn_c + trp_L_e
r2030: orn_e + phe_L_c --> orn_c + phe_L_e
r2031: orn_e + tyr_L_c --> orn_c + tyr_L_e
r2032: cys_L_c + orn_e --> cys_L_e + orn_c
r2034: orn_e + pro_L_c --> orn_c + pro_L_e
r2035: asn_L_c + orn_e --> asn_L_e + orn_c
r2036: orn

r2186: HC02197_e + HC02201_c + hco3_c <=> HC02197_c + HC02201_e + hco3_e
r2187: HC02198_e + HC02199_c + hco3_c <=> HC02198_c + HC02199_e + hco3_e
r2188: HC02198_e + HC02200_c + hco3_c <=> HC02198_c + HC02200_e + hco3_e
r2189: HC02198_e + HC02201_c + hco3_c <=> HC02198_c + HC02201_e + hco3_e
r2190: HC02199_c + HC02220_e + hco3_c <=> HC02199_e + HC02220_c + hco3_e
r2191: HC02200_c + HC02220_e + hco3_c <=> HC02200_e + HC02220_c + hco3_e
r2192: HC02201_c + HC02220_e + hco3_c <=> HC02201_e + HC02220_c + hco3_e
r2193: bilirub_e + gthrd_c + hco3_c <=> bilirub_c + gthrd_e + hco3_e
r2194: gthrd_c + hco3_c + thyox_L_e <=> gthrd_e + hco3_e + thyox_L_c
r2195: gthrd_c + hco3_c + triodthy_e <=> gthrd_e + hco3_e + triodthy_c
r2196: HC02187_e + gthrd_c + hco3_c <=> HC02187_c + gthrd_e + hco3_e
r2197: gthrd_c + hco3_c + prostge2_e <=> gthrd_e + hco3_e + prostge2_c
r2198: gthrd_c + hco3_c + leuktrC4_e <=> gthrd_e + hco3_e + leuktrC4_c
r2199: bildglcur_e + gthrd_c + hco3_c <=> bildglcur_c + gthrd_e + hco

r2306: HC02200_c + hco3_c + prostgi2_e <=> HC02200_e + hco3_e + prostgi2_c
r2307: HC02201_c + hco3_c + prostgi2_e <=> HC02201_e + hco3_e + prostgi2_c
r2308: gthrd_c + hco3_c + hista_e <=> gthrd_e + hco3_e + hista_c
r2309: gthrd_c + hco3_c + nrpphr_e <=> gthrd_e + hco3_e + nrpphr_c
r2310: gthrd_c + hco3_c + srtn_e <=> gthrd_e + hco3_e + srtn_c
r2311: ach_e + gthrd_c + hco3_c <=> ach_c + gthrd_e + hco3_e
r2312: dopa_e + gthrd_c + hco3_c <=> dopa_c + gthrd_e + hco3_e
r2313: arachd_e + gthrd_c + hco3_c <=> arachd_c + gthrd_e + hco3_e
r2314: gthrd_c + hco3_c + leuktrD4_e <=> gthrd_e + hco3_e + leuktrD4_c
r2315: gthrd_c + hco3_c + wharachd_e <=> gthrd_e + hco3_e + wharachd_c
r2316: gthrd_c + hco3_c + leuktrA4_e <=> gthrd_e + hco3_e + leuktrA4_c
r2317: gthrd_c + hco3_c + leuktrB4_e <=> gthrd_e + hco3_e + leuktrB4_c
r2318: gthrd_c + hco3_c + leuktrF4_e <=> gthrd_e + hco3_e + leuktrF4_c
r2319: HC02202_e + gthrd_c + hco3_c <=> HC02202_c + gthrd_e + hco3_e
r2320: HC02203_e + gthrd_c + hco3_c <=> 

In [6]:
model

Name,iCHOxxxx
Memory address,1204762c0
Number of metabolites,5469
Number of reactions,11283
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [7]:
#Create stoichiometric matrix for the model

from cobra.util import create_stoichiometric_matrix

S = create_stoichiometric_matrix(model, array_type = 'DataFrame')

In [8]:
S

,10FTHF5GLUtl,10FTHF5GLUtm,10FTHF6GLUtl,10FTHF6GLUtm,10FTHF7GLUtl,10FTHF7GLUtm,10FTHFtl,10FTHFtm,11DOCRTSLtm,11DOCRTSLtr,...,r2520,r2521,r2525,r2526,r2532,r2534,r2535,r2537,r2538,r2539
10fthf5glu_c,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10fthf5glu_l,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10fthf5glu_m,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10fthf6glu_c,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10fthf6glu_l,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xylnact_D_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xylt_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zn2_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C02470_m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
cor = S.iloc[:,0:10000].corr()

In [56]:
cor =  cor[cor == 1]

In [57]:
s = cor.unstack()
s = s.sort_values(kind="quicksort")

In [58]:
for (a,b) in zip(s,s.index):
    if (b[0] != b[1]) and a == 1:
        print(b)

('PA_CHOter', 'PA_HSter')
('PA_CHOtg', 'PA_HStg')
('PA_CHOtn', 'PA_HStn')
('PA_HSter', 'PA_CHOter')
('PA_HStg', 'PA_CHOtg')
('PALCOACAROPLd_r', 'ARCOAATr')
('PALCOACAROPLc_x', 'ARCOAATp')
('PALCOACAROPLc_r', 'PMTCOAPTr')
('PAIL4P_CHOtn', 'PAIL4P_HStn')
('PAIL4P_HStn', 'PAIL4P_CHOtn')
('PAIL_CHOtn', 'PAIL_HStn')
('PAIL_HStn', 'PAIL_CHOtn')
('PA_HStn', 'PA_CHOtn')
('PALCOACAROPL_r', 'PLMCOAPTr')
('PALCOACAROPL_x', 'LNCOALTp')
('PALCOACAROPLa_r', 'LNCOALTr')
('PALCOACAROPLb_x', 'PMTCOAPTp')
('PAIL45P_HStn', 'PAIL45P_CHOtn')
('PCt', 'PCtabc')
('PCtabc', 'PCt')
('PCHOL_CHOter', 'PCHOL_HSter')
('PCHOL_CHOtg', 'PCHOL_HStg')
('PCHOL_HStg', 'PCHOL_CHOtg')
('PCHOL_HSter', 'PCHOL_CHOter')
('PAIL45P_CHOtn', 'PAIL45P_HStn')
('OXASOXIRCC_c', 'r0082')
('OXASOXIRCC_m', 'r0083')
('OXASOXIRCC_x', 'r0084')
('OXYODPCm_e', 'FTMT')
('OXYODPCm_e', 'r0016')
('OXAPItm', 'MC_TCDBe3')
('ORNHIStm', 'MC_TCDBf14')
('ORNLYStm', 'MC_TCDBf6')
('ORNt', 'ORNtiDF')
('OROTte', 'PostulatedtRXN2')
('OXAICITtm', 'MC_TCDBd10'

1.0

In [ ]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))
    break

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))
    break

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

In [ ]:
# this functions are for fetching information from the BiGG database

def get_rxninfo(rxn):

    #download the page
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn)
    if response.status_code != 200:
        print('Status code:', response.status_code,f'Failed to fetch info on {rxn} from iCHOv1 model')
        response=session.get('http://bigg.ucsd.edu/universal/reactions/'+rxn)
        if response.status_code != 200:
            print('Status code:', response.status_code,f'Failed to fetch info on {rxn} from BiGG')

    #parse using beautiful soup
    rxn_doc = BeautifulSoup(response.text,'html.parser')

    return rxn_doc


def bigg_attributes(rxn):
    
    '''
    recieves a rxn file from the get_rxninfo function
    and returns Reaction Description, Subsystem
    '''
    if rxn.title.get_text().endswith('iCHOv1'):
        # Reaction description
        rxn_d = rxn.find_all("p")[0].get_text()

        # Reaction formula
        form = rxn.find_all("p")[2].get_text()

        # Subsystem
        subsystem = rxn.find_all("p")[5].get_text()
    else:
        # Reaction description
        rxn_d = rxn.find_all("p")[0].get_text()

        # Reaction formula
        form = rxn.find_all("p")[1].get_text()

        # Subsystem
        subsystem = ''
    
    return rxn_d, form, subsystem


In [ ]:
#all_dfs3 = pd.DataFrame(columns = ['Reactions', 'Reaction Description', 'Reaction Formula', 'Subsystem'])
att = []
for i,r in enumerate(tqdm((all_dfs2.index))):
    try:
        rxn = get_rxninfo(r)
        rxn_d, form, subsystem = bigg_attributes(rxn)
        print(rxn_d, form, subsystem)
        if all_dfs2['Reaction Formula'][i] == None:
            all_dfs2['Reaction Formula'][i] = form
        if all_dfs2['Reaction Name'][i] == None:
            all_dfs2['Reaction Name'][i] = rxn_d
    except:
        print(f'Could not find info for {r}')
        break
        
all_dfs2.to_excel('../Data/all_dfs2.xlsx')

In [ ]:
rxn.find_all("p")

In [ ]:
rxn.find_all("p")[2].get_text()

In [ ]:
rxn.find_all("div", class_="col-lg-8")

In [ ]:
rxn = get_rxninfo('5G2OXPTtx')
print(rxn.title.get_text().endswith('iCHOv1'))


In [ ]:
rxn.find_all("p")[3].get_text()

In [ ]:
for i,r in enumerate(tqdm((all_dfs2.index))):
    if r.endswith('_cho'):
        print(r,i)
        if all_dfs2.index[i-1] == r.split('_')[0]:
            print(all_dfs2.index[i-1],i-1)
        #print(all_dfs2.index[i-1],i-1)

In [ ]:
#def f(x):
#    d = {}
#    
#    l = []
#    for x in x['GPR']:
#        l.append(x)
#    if len(set(l)) == 1:
#        print(np.unique(l))
#        d['GPR'] = np.unique(l)
#    elif len(set(l)) > 1:
#        print(x)
#        d['GPR'] = x
#        
#    return pd.Series(d, index=['GPR'])#, 'a_max', 'b_mean', 'c_d_prodsum'])


#all_dfs1['GPR'] = all_dfs.groupby(['Reaction'], group_keys=True).apply(f)
#all_dfs1
#all_dfs1.to_excel('../Data/all_dfs1.xlsx')